In [1]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline
from peft import PeftModel

/home/ujx4ab/ondemand/NLP_Final_Political_Bias_Shifts/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

trained_model_path = '/home/ujx4ab/ondemand/NLP_Final_Political_Bias_Shifts/fine_tuned_llms/FacebookAI_roberta-large/checkpoints/CNN_model_3_a_5_epoch/2024-12-15 19-48-53/checkpoint-750'
base_model_path = '/home/ujx4ab/ondemand/NLP_Final_Political_Bias_Shifts/src/Local Models/FacebookAI_roberta-large'
lora_weights_path = f"{trained_model_path}/adapter_model.safetensors"

# Load base model and tokenizer
base_model = AutoModelForMaskedLM.from_pretrained(base_model_path, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Load base model and tokenizer
ft_base_model = AutoModelForMaskedLM.from_pretrained(trained_model_path, torch_dtype=torch.float16, device_map="auto")

base_pipe = pipeline(
    "fill-mask", 
    model=base_model, 
    tokenizer=tokenizer,
    torch_dtype=torch.float16, 
    device_map="auto"
)

trained_pipe = pipeline(
    "fill-mask", 
    model=ft_base_model, 
    tokenizer=tokenizer,
    torch_dtype=torch.float16, 
    device_map="auto"
)


Device set to use cuda:0
Device set to use cuda:0


In [7]:
cloze_statement = 'I think gun ownership is <mask>'
#"Gun ownership does <mask> to increase safety by allowing law-abiding citizens to protect themselves "


In [12]:
base_pipe(cloze_statement)

[{'score': 0.07879638671875,
  'token': 734,
  'token_str': '...',
  'sequence': 'I think gun ownership is...'},
 {'score': 0.0653076171875,
  'token': 505,
  'token_str': ' important',
  'sequence': 'I think gun ownership is important'},
 {'score': 0.0408935546875,
  'token': 1593,
  'token_str': ' wrong',
  'sequence': 'I think gun ownership is wrong'},
 {'score': 0.0408935546875,
  'token': 1174,
  'token_str': '…',
  'sequence': 'I think gun ownership is…'},
 {'score': 0.038421630859375,
  'token': 205,
  'token_str': ' good',
  'sequence': 'I think gun ownership is good'}]

In [13]:
trained_pipe(cloze_statement)

[{'score': 0.06304931640625,
  'token': 1593,
  'token_str': ' wrong',
  'sequence': 'I think gun ownership is wrong'},
 {'score': 0.0506591796875,
  'token': 12103,
  'token_str': ' stupid',
  'sequence': 'I think gun ownership is stupid'},
 {'score': 0.046112060546875,
  'token': 2702,
  'token_str': ' dangerous',
  'sequence': 'I think gun ownership is dangerous'},
 {'score': 0.0279693603515625,
  'token': 2439,
  'token_str': ' illegal',
  'sequence': 'I think gun ownership is illegal'},
 {'score': 0.024688720703125,
  'token': 2051,
  'token_str': ' fine',
  'sequence': 'I think gun ownership is fine'}]